In [ ]:
#install what needs to be installed setting up instance here

#Google Drive mount here
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
#set up file location here
import sys
sys.path.append('/content/gdrive/LLMHallucination')

# import libraries
from Util import load_data_JSON, load_model, write_out
import pandas as pd
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

In [ ]:
#load the data
general_df, prompt_g, correct_g, gtp4_g, Info_g = load_data_JSON('input/general_data.json', 'general')
qa_df, prompt_q, correct_q, gtp4_q, Info_q = load_data_JSON('input/qa.json', 'qa')
sum_df, prompt_s, correct_s, gtp4_s, Info_s = load_data_JSON('input/summarization_data.json', 'sum')

In [ ]:
# Access input (API key, etc.)
access = input('Access code?')

#setup of model
tokenizer, model = load_model("meta-llama/Llama-2-7b-chat-hf", access)

In [ ]:
# ChainPoll method
def chainpoll(model, tokenizer, prompts, num_responses=5):
    # Function to detect hallucinations using ChainPoll
    out_q = []

    # Process the questions (currently the first 2 questions)
    for i, prompt in enumerate(prompts[:2]):
        hallucination_scores = []

        # Run 5 times for the ChainPoll method
        for _ in range(num_responses):
            # Tokenize the prompt for input
            inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

            # Generate the response from the model
            with torch.no_grad():
                outputs = model.generate(
                    inputs['input_ids'],
                    max_length=50,  # Adjust based on needs
                    num_beams=5,
                    early_stopping=True
                )

            # Decode the output response
            response = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Ask the model if there are hallucinations using the CoT method
            hallucination_prompt = f"Does the following ouput contain hallucinations? Explain in detail:\n\nOutput: {response}\n"
            hallucination_inputs = tokenizer(hallucination_prompt, return_tensors="pt").to("cuda")

            # Generate the hallucination response
            with torch.no_grad():
                hall_outputs = model.generate(
                    hallucination_inputs['input_ids'],
                    max_length=100,  # Adjust this as needed to allow space for CoT
                    num_beams=5,
                    early_stopping=True
                )

            # Decode the hallucination response
            hallucination_response = tokenizer.decode(hall_outputs[0], skip_special_tokens=True)

            # Check for "yes" or "no" in the hallucination response and store the score
            if "yes" in hallucination_response.lower():
                hallucination_scores.append(1)
            else:
                hallucination_scores.append(0)

        # Compute final hallucination score as the proportion of "yes" responses
        hallucination_score = sum(hallucination_scores) / num_responses

        # Save the final hallucination judgment and explanation
        out_q.append({
            "prompt": prompt,
            "response": response,
            "hallucination_score": hallucination_score,
            "explanation": hallucination_response
        })

    # Write the results
    write_out("ChainPoll_qa_results.csv", out_q)

In [ ]:
# Call the ChainPoll method
chainpoll(model, tokenizer, prompt_q)

In [ ]:
#run of the prompts for the general section
out_g = []
for p in prompt_g:
    # Tokenize the prompt
    inputs = tokenizer(p, return_tensors="pt").to("cuda")

    # Generate the response
    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            max_length=50,  # Adjust this based on your needs
            num_beams=5,  # Optional: Use beam search to improve generation quality
            early_stopping=True
        )

    # Decode the output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    out_g.append(response)

write_out("Baseline_control_general.csv", out_g)

In [ ]:
#run of the prompts for summarization
out_s = []
for p in prompt_s:
    # Tokenize the prompt
    inputs = tokenizer("Summarize this document: " + p, return_tensors="pt").to("cuda")

    # Generate the response
    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            max_length=500,  # Adjust this based on your needs
            num_beams=5,  # Optional: Use beam search to improve generation quality
            early_stopping=True
        )

    # Decode the output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    out_s.append(response)

write_out("Baseline_control_summarisation.csv", out_s)